# MMI prediction

## 5. Training
- Shortlist 10 models
- Hyperparameter tuning top 3 + LR
- Fit tuned models on entire training set and save

In [1]:
from datetime import datetime
from tqdm import tqdm
import numpy as np
import pandas as pd
import pickle
import joblib

from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import make_pipeline as make_pipeline_sk
from sklearn.inspection import permutation_importance

import xgboost as xgb
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE

from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.metrics import AUC
from keras.wrappers.scikit_learn import KerasClassifier

In [2]:
xgb.config_context(verbosity = 0)

In [3]:
# load and shuffle data

X_train = pd.read_pickle('X_train_trans.pkl')
y_train = np.ravel(pd.read_pickle('y_train.pkl'))
X_train, y_train = shuffle(X_train, y_train)
print('There are {} training samples'.format(X_train.shape[0]))

There are 304 training samples


## Shortlist several models

In [33]:
# create dictionaries of hyperparameters for sklearn and GBT models

max_iter = 1000

log_reg_params = [{'penalty': 'none', 'max_iter': max_iter}]
dec_tree_params = [{'criterion': 'gini'}, {'criterion': 'entropy'}]
rand_for_params = [{'criterion': 'gini'}, {'criterion': 'entropy'}]
kneighbors_params = [{'n_neighbors': 3}, {'n_neighbors': 5}]
naive_bayes_params = [{}]
svc_params = [{'C': 0.01}, {'C': 0.1}, {'C': 1}, {'C': 10}]
xgb_params = [{'use_label_encoder': False}]
cb_params = [{'verbose': False}]
lgbm_params = [{}]
mlp_params = [{'hidden_layer_sizes': (10,), 'max_iter': max_iter}, {'hidden_layer_sizes': (10, 10,), 'max_iter': max_iter}, 
              {'hidden_layer_sizes': (10, 10, 10,), 'max_iter': max_iter}]

models = [
    ['log regression', LogisticRegression, log_reg_params],
    ['decision tree', DecisionTreeClassifier, dec_tree_params],
    ['random forest', RandomForestClassifier, rand_for_params],
    ['k neighbors', KNeighborsClassifier, kneighbors_params],
    ['naive bayes', GaussianNB, naive_bayes_params],
    ['support vector machines', SVC, svc_params],
    ['XG boost', xgb.XGBClassifier, xgb_params],
    ['Cat boost', CatBoostClassifier, cb_params],
    ['Light GBM', LGBMClassifier, lgbm_params],
    ['MLP', MLPClassifier, mlp_params]
]

In [13]:
# helper function to evaluate sklearn and GBT models

def eval_models(models, score, X_train, y_train, fname):
    results = []
    result_file = open(fname, 'a')
    
    for model_name, Model, params_list in models:
        for params in params_list:
            model = make_pipeline(
                SMOTE(),
                Model(**params)
            )
            cv = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 10)
            scores = list(cross_val_score(model, X_train, y_train, scoring = score, cv = cv))
            results.append((model_name, model, params, np.mean(scores), np.std(scores), scores))
    
    results.sort(key = lambda x:x[-3], reverse = True)
    
    # write score summary to txt file
    result_file.write('\nmean {} scores:\n\n'.format(score))
    for modelname, model, params, mean, std, scores in results:
        result_file.write(str(modelname) + '\t' + str(params) + '\t' + str(mean) + '\n')
    result_file.close()
    
    # write scores to dataframe
    df = pd.DataFrame()
    for modelname, model, params, mean, std, scores in results:
        column_name = str(modelname) + str(params)
        df[column_name] = scores
    df.to_pickle('experiments/init_training_results_{}.pkl'.format(score))
    
    # write permutation feature importances to dataframe
    if score == 'roc_auc':
        for modelname, model, params, mean, std, scores in results:
            model.fit(X_train, y_train)
            feat_imp = permutation_importance(model, X_train, y_train, n_repeats = 10)
            feat_imp_df = pd.DataFrame(feat_imp.importances, index = X_train.columns.tolist())
            feat_imp_df.to_pickle('experiments/init_training_feat_imp_{}.pkl'.format(modelname).replace(' ', ''))

In [35]:
# evaluate sklearn/GBT models

scores = ['roc_auc', 'precision', 'recall', 'f1', 'accuracy']
fname = 'experiments/init_training_summary' + '_' + str(datetime.now().year) + '_' + str(datetime.now().month) \
    + '_' + str(datetime.now().day) + '_' + str(datetime.now().hour) + '_' + \
    str(datetime.now().minute) + '.txt'
        
for score in scores:
    eval_models(models = models, score = score, X_train = X_train, y_train = y_train, fname = fname)

[16:25:19] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:25:19] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:25:20] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:25:20] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the o

[16:25:26] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:25:26] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:25:26] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:25:26] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the o

/Users/haydnhoffman/ml/my_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/haydnhoffman/ml/my_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/haydnhoffman/ml/my_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/haydnhoffman/ml/my_env/lib/python3.9/site-packages/sklearn/metrics

/Users/haydnhoffman/ml/my_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/haydnhoffman/ml/my_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/haydnhoffman/ml/my_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[16:34:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:34:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:34:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:34:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the o

...

[16:40:52] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:40:52] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:40:52] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:40:52] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the o

[16:46:48] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:46:48] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:46:48] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:46:48] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the o

[16:46:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:46:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:46:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:46:55] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the o

[16:52:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:52:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:52:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:52:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the o

[16:52:59] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:53:00] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:53:00] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


## Hyperparameter tuning

In [50]:
# nested CV to find best parameters for sklearn models

def eval_params(fname, tuning_model, param_grid):
    
    results = []
    result_file = open(fname, 'a')

    skf = StratifiedKFold(n_splits = 5, shuffle = True)
    fold_no = 1
    
    for train_index, test_index in skf.split(X_train, y_train):

        X_train_split, X_test = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_split, y_test = y_train[train_index], y_train[test_index]
        
        # find best model params
        print('finding best model parameters for fold number {}'.format(fold_no))
        model = make_pipeline(
            SMOTE(),
            tuning_model
        )
        grid = GridSearchCV(estimator = model, param_grid = param_grid, n_jobs = -1, cv = 5, error_score = 'raise', 
                           scoring = 'roc_auc', verbose = 3)
        grid_result = grid.fit(X_train_split, y_train_split)
        
        # evaluate best model params on outer fold
        print('evaluating model for fold number {}'.format(fold_no))
        best_params = grid_result.best_params_
        print(best_params)
        best_model = make_pipeline_sk(tuning_model)
        best_model.set_params(**best_params)
        best_model.fit(X_train_split, y_train_split)
        score = roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1])
        results.append((best_params, score))
        print('parameters: {}'.format(str(best_params)))
        print('AUC score: {}'.format(score))
        fold_no += 1

    results.sort(key = lambda x:x[-1], reverse = True)
    result_file.write('\nAUC scores:\n\n')
    
    # write score summary to text file
    for best_params, score in results:
        result_file.write(str(best_params) + '\t' + str(score) + '\n')
    result_file.close()
    
    # write scores to dataframe:
    df = pd.DataFrame(results, columns = ['params', 'score'])
    df.to_pickle('experiments/hyperparameter_tuning_results_{}.pkl'.format(tuning_model).replace(' ', ''))

In [ ]:
# define param grid for grid search - random forest

n_estimators = [int(x) for x in range(20, 200, 20)] 
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10] 
min_samples_leaf = [1, 2, 4]
rf_grid = {'randomforestclassifier__n_estimators': n_estimators, 
          'randomforestclassifier__max_features': max_features, 
          'randomforestclassifier__max_depth': max_depth, 
          'randomforestclassifier__min_samples_split': min_samples_split, 
          'randomforestclassifier__min_samples_leaf': min_samples_leaf}

In [39]:
# tune random forest

fname = 'experiments/rf_tuning' + '_' + str(datetime.now().year) + '_' + str(datetime.now().month) \
    + '_' + str(datetime.now().day) + '_' + str(datetime.now().hour) + '_' + \
    str(datetime.now().minute) + '.txt'

eval_params(fname = fname, tuning_model = RandomForestClassifier(), param_grid = rf_grid)

finding best model parameters for fold number 1
Fitting 5 folds for each of 1944 candidates, totalling 9720 fits
evaluating model for fold number 1
{'randomforestclassifier__max_depth': 60, 'randomforestclassifier__max_features': 'auto', 'randomforestclassifier__min_samples_leaf': 1, 'randomforestclassifier__min_samples_split': 2, 'randomforestclassifier__n_estimators': 60}
parameters: {'randomforestclassifier__max_depth': 60, 'randomforestclassifier__max_features': 'auto', 'randomforestclassifier__min_samples_leaf': 1, 'randomforestclassifier__min_samples_split': 2, 'randomforestclassifier__n_estimators': 60}
AUC score: 0.7488207547169812
finding best model parameters for fold number 2
Fitting 5 folds for each of 1944 candidates, totalling 9720 fits
evaluating model for fold number 2
{'randomforestclassifier__max_depth': None, 'randomforestclassifier__max_features': 'auto', 'randomforestclassifier__min_samples_leaf': 2, 'randomforestclassifier__min_samples_split': 5, 'randomforestclas

In [48]:
# define param grid for grid search - SVM

C = np.logspace(-2, 3, num = 6)
gamma = np.logspace(-4, 0, num = 5)
kernel = ['rbf']
probability = [True]

svm_grid = {'svc__C': C, 
           'svc__gamma': gamma, 
           'svc__kernel': kernel,
           'svc__probability': probability}

In [49]:
# tune SVM

fname = 'experiments/svm_tuning' + '_' + str(datetime.now().year) + '_' + str(datetime.now().month) \
    + '_' + str(datetime.now().day) + '_' + str(datetime.now().hour) + '_' + \
    str(datetime.now().minute) + '.txt'

eval_params(fname = fname, tuning_model = SVC(), param_grid = svm_grid)

finding best model parameters for fold number 1
Fitting 5 folds for each of 30 candidates, totalling 150 fits
evaluating model for fold number 1
{'svc__C': 1000.0, 'svc__gamma': 0.0001, 'svc__kernel': 'rbf', 'svc__probability': True}
parameters: {'svc__C': 1000.0, 'svc__gamma': 0.0001, 'svc__kernel': 'rbf', 'svc__probability': True}
AUC score: 0.8089622641509434
finding best model parameters for fold number 2
Fitting 5 folds for each of 30 candidates, totalling 150 fits
evaluating model for fold number 2
{'svc__C': 0.1, 'svc__gamma': 0.0001, 'svc__kernel': 'rbf', 'svc__probability': True}
parameters: {'svc__C': 0.1, 'svc__gamma': 0.0001, 'svc__kernel': 'rbf', 'svc__probability': True}
AUC score: 0.8160377358490566
finding best model parameters for fold number 3
Fitting 5 folds for each of 30 candidates, totalling 150 fits
evaluating model for fold number 3
{'svc__C': 0.01, 'svc__gamma': 0.001, 'svc__kernel': 'rbf', 'svc__probability': True}
parameters: {'svc__C': 0.01, 'svc__gamma': 0.

In [57]:
# define param grid for grid search - log reg

penalty = ['l2', 'none']
C = np.logspace(-2, 2, num = 5)
max_iter = [1000]

logreg_grid = {'logisticregression__penalty': penalty, 
              'logisticregression__C': C, 
              'logisticregression__max_iter': max_iter}

In [58]:
# tune logistic regression

fname = 'experiments/logreg_tuning' + '_' + str(datetime.now().year) + '_' + str(datetime.now().month) \
    + '_' + str(datetime.now().day) + '_' + str(datetime.now().hour) + '_' + \
    str(datetime.now().minute) + '.txt'

eval_params(fname = fname, tuning_model = LogisticRegression(), param_grid = logreg_grid)

finding best model parameters for fold number 1
Fitting 5 folds for each of 10 candidates, totalling 50 fits
evaluating model for fold number 1
{'logisticregression__C': 0.1, 'logisticregression__max_iter': 1000, 'logisticregression__penalty': 'l2'}
parameters: {'logisticregression__C': 0.1, 'logisticregression__max_iter': 1000, 'logisticregression__penalty': 'l2'}
AUC score: 0.8726415094339622
finding best model parameters for fold number 2
Fitting 5 folds for each of 10 candidates, totalling 50 fits


/Users/haydnhoffman/ml/my_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/haydnhoffman/ml/my_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


evaluating model for fold number 2
{'logisticregression__C': 100.0, 'logisticregression__max_iter': 1000, 'logisticregression__penalty': 'none'}
parameters: {'logisticregression__C': 100.0, 'logisticregression__max_iter': 1000, 'logisticregression__penalty': 'none'}
AUC score: 0.6320754716981132
finding best model parameters for fold number 3
Fitting 5 folds for each of 10 candidates, totalling 50 fits
evaluating model for fold number 3
{'logisticregression__C': 0.1, 'logisticregression__max_iter': 1000, 'logisticregression__penalty': 'l2'}
parameters: {'logisticregression__C': 0.1, 'logisticregression__max_iter': 1000, 'logisticregression__penalty': 'l2'}
AUC score: 0.8018867924528302
finding best model parameters for fold number 4
Fitting 5 folds for each of 10 candidates, totalling 50 fits
evaluating model for fold number 4
{'logisticregression__C': 0.1, 'logisticregression__max_iter': 1000, 'logisticregression__penalty': 'l2'}
parameters: {'logisticregression__C': 0.1, 'logisticreg

In [15]:
# # define param grid for grid search - catboost

# iterations = [5, 10, 15, 20, 25, 50, 100]
# learning_rate = [0.01, 0.05, 0.1]
# depth = np.arange(3, 15, 2)
# cb_grid = {'catboostclassifier__iterations': iterations,
#            'catboostclassifier__learning_rate': learning_rate,
#            'catboostclassifier__depth': depth}

In [11]:
# # nested CV to find best parameters for catboost

# def eval_cb_params(fname, param_grid):
    
#     results = []
#     result_file = open(fname, 'a')

#     skf = StratifiedKFold(n_splits = 5, shuffle = True)
#     fold_no = 1
    
#     for train_index, test_index in skf.split(X_train, y_train):

#         X_train_split, X_test = X_train.iloc[train_index], X_train.iloc[test_index]
#         y_train_split, y_test = y_train[train_index], y_train[test_index]
        
#         # find best model params
#         print('finding best model parameters for fold number {}'.format(fold_no))
#         tuning_model = CatBoostClassifier()
#         model = make_pipeline(
#             SMOTE(),
#             tuning_model
#         )
#         grid = GridSearchCV(estimator = model, param_grid = param_grid, n_jobs = -1, cv = 5, error_score = 'raise', 
#                            scoring = 'roc_auc', verbose = 3)
#         grid_result = grid.fit(X_train_split, y_train_split)
        
#         # evaluate best model params on outer fold
#         print('evaluating model for fold number {}'.format(fold_no))
#         best_params = grid_result.best_params_
#         print(best_params)
#         best_model = make_pipeline_sk(tuning_model)
#         best_model.set_params(**best_params)
#         best_model.fit(X_train_split, y_train_split)
#         score = roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1])
#         results.append((best_params, score))
#         print('parameters: {}'.format(str(best_params)))
#         print('AUC score: {}'.format(score))
#         fold_no += 1

#     results.sort(key = lambda x:x[-1], reverse = True)
#     result_file.write('\nAUC scores:\n\n')
#     for best_params, score in results:
#         result_file.write(str(best_params) + '\t' + str(score) + '\n')
#     result_file.close()

In [16]:
# # tune catboost

# fname = 'experiments/cb_tuning' + '_' + str(datetime.now().year) + '_' + str(datetime.now().month) \
#     + '_' + str(datetime.now().day) + '_' + str(datetime.now().hour) + '_' + \
#     str(datetime.now().minute) + '.txt'

# eval_cb_params(fname = fname, param_grid = cb_grid)

finding best model parameters for fold number 1
Fitting 5 folds for each of 126 candidates, totalling 630 fits
0:	learn: 0.6013844	total: 6.57ms	remaining: 59.2ms
1:	learn: 0.5396119	total: 12.9ms	remaining: 51.6ms
2:	learn: 0.4912861	total: 16ms	remaining: 37.4ms
3:	learn: 0.4378666	total: 18.6ms	remaining: 27.9ms
4:	learn: 0.4092037	total: 21.2ms	remaining: 21.2ms
5:	learn: 0.3703096	total: 23.7ms	remaining: 15.8ms
6:	learn: 0.3430178	total: 26.2ms	remaining: 11.2ms
7:	learn: 0.3142624	total: 28.8ms	remaining: 7.21ms
8:	learn: 0.2982096	total: 31.3ms	remaining: 3.48ms
9:	learn: 0.2789672	total: 34.1ms	remaining: 0us
evaluating model for fold number 1
{'catboostclassifier__depth': 5, 'catboostclassifier__iterations': 10, 'catboostclassifier__learning_rate': 0.1}
0:	learn: 0.6138371	total: 3.95ms	remaining: 35.6ms
1:	learn: 0.5463525	total: 10ms	remaining: 40.1ms
2:	learn: 0.4992641	total: 12.2ms	remaining: 28.6ms
3:	learn: 0.4616242	total: 14.2ms	remaining: 21.4ms
4:	learn: 0.4096487	

5:	learn: 0.6432553	total: 32.6ms	remaining: 511ms
6:	learn: 0.6373101	total: 41.4ms	remaining: 551ms
7:	learn: 0.6280618	total: 46.2ms	remaining: 531ms
8:	learn: 0.6193380	total: 47.3ms	remaining: 478ms
9:	learn: 0.6124351	total: 48.2ms	remaining: 434ms
10:	learn: 0.6059303	total: 49.2ms	remaining: 398ms
11:	learn: 0.5977625	total: 50.1ms	remaining: 368ms
12:	learn: 0.5922348	total: 51.1ms	remaining: 342ms
13:	learn: 0.5847601	total: 52.1ms	remaining: 320ms
14:	learn: 0.5797964	total: 53ms	remaining: 300ms
15:	learn: 0.5739309	total: 54ms	remaining: 283ms
16:	learn: 0.5688211	total: 55ms	remaining: 268ms
17:	learn: 0.5629417	total: 55.9ms	remaining: 255ms
18:	learn: 0.5581913	total: 56.8ms	remaining: 242ms
19:	learn: 0.5538075	total: 59.4ms	remaining: 238ms
20:	learn: 0.5461016	total: 60.3ms	remaining: 227ms
21:	learn: 0.5402952	total: 61.3ms	remaining: 217ms
22:	learn: 0.5350369	total: 62.2ms	remaining: 208ms
23:	learn: 0.5301378	total: 63.2ms	remaining: 200ms
24:	learn: 0.5227813	to

parameters: {'catboostclassifier__depth': 3, 'catboostclassifier__iterations': 50, 'catboostclassifier__learning_rate': 0.05}
AUC score: 0.8416149068322981
finding best model parameters for fold number 5
Fitting 5 folds for each of 126 candidates, totalling 630 fits
0:	learn: 0.6578717	total: 9.06ms	remaining: 172ms
1:	learn: 0.6309444	total: 12.1ms	remaining: 108ms
2:	learn: 0.6035201	total: 13.3ms	remaining: 75.4ms
3:	learn: 0.5834612	total: 14.5ms	remaining: 58ms
4:	learn: 0.5611351	total: 15.7ms	remaining: 47.1ms
5:	learn: 0.5368898	total: 16.9ms	remaining: 39.4ms
6:	learn: 0.5247337	total: 18.1ms	remaining: 33.5ms
7:	learn: 0.5050687	total: 19.2ms	remaining: 28.8ms
8:	learn: 0.4973934	total: 20.4ms	remaining: 25ms
9:	learn: 0.4773655	total: 21.7ms	remaining: 21.7ms
10:	learn: 0.4623923	total: 22.9ms	remaining: 18.7ms
11:	learn: 0.4472013	total: 24.1ms	remaining: 16.1ms
12:	learn: 0.4357787	total: 25.3ms	remaining: 13.6ms
13:	learn: 0.4224886	total: 26.6ms	remaining: 11.4ms
14:	lea

## Hyperparameter tuning - Keras models

In [6]:
# helper function to build a keras DNN model

def build_model(input_shape = X_train.shape[1], layer_size_factor = 3, num_hidden_layers = 1, dropout_rate = 0):
    
    inputs = Input(shape = input_shape)
    x = Dense(np.power(2, layer_size_factor), activation = 'relu')(inputs)
    x = Dropout(dropout_rate)(x)
    if num_hidden_layers > 1:
        for i in range(num_hidden_layers - 1):
            x = Dense(np.power(2, layer_size_factor), activation = 'relu')(x)
            x = Dropout(dropout_rate)(x)
    outputs = Dense(1, activation = 'sigmoid')(x)
    model = Model(inputs = inputs, outputs = outputs)
    model.compile('adam', 'binary_crossentropy', metrics = [AUC()])
    
    return model

In [61]:
# define param grid for grid search

sizes = np.arange(3, 9)
layers = np.arange(1, 4) 
rates = np.arange(0, 0.6, 0.1)
batch_size = [40, 60, 80, 100]
epochs = [10, 50, 100] 

keras_grid = dict(kerasclassifier__layer_size_factor = sizes, 
                 kerasclassifier__num_hidden_layers = layers, 
                 kerasclassifier__dropout_rate = rates, 
                 kerasclassifier__batch_size = batch_size, 
                 kerasclassifier__epochs = epochs)

In [63]:
# nested CV to find best parameters for DNN model

def eval_keras_params(fname, param_grid):

    results = []
    result_file = open(fname, 'a')

    skf = StratifiedKFold(n_splits = 5, shuffle = True)
    fold_no = 1
    
    for train_index, test_index in skf.split(X_train, y_train):

        X_train_split, X_test = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_split, y_test = y_train[train_index], y_train[test_index]

        # find best model params
        print('finding best model parameters for fold number {}'.format(fold_no))
        model = make_pipeline(
            SMOTE(),
            KerasClassifier(build_fn = build_model, input_shape = X_train_split.shape[1]) # wrapper for keras model
        )
        grid = GridSearchCV(estimator = model, param_grid = param_grid, n_jobs = -1, cv = 5, error_score = 'raise', 
                           scoring = 'roc_auc', verbose = 2)
        grid_result = grid.fit(X_train_split, y_train_split)
    
        # evaluate best model params on outer fold
        print('evaluating model for fold number {}'.format(fold_no))
        best_params = grid_result.best_params_
        best_model = make_pipeline_sk(KerasClassifier(build_fn = build_model, input_shape = X_train_split.shape[1]))
        best_model.set_params(**best_params)
        best_model.fit(X_train_split, y_train_split)
        score = roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1])
        results.append((best_params, score))
        print('parameters: {}'.format(str(best_params)))
        print('AUC score: {}'.format(score))
        fold_no += 1

    results.sort(key = lambda x:x[-1], reverse = True)
    result_file.write('\nAUC scores:\n\n')
    
    # write score summary to text file
    for best_params, score in results:
        result_file.write(str(best_params) + '\t' + str(score) + '\n')
    result_file.close()
    
    # write scores to dataframe:
    df = pd.DataFrame(results, columns = ['params', 'score'])
    df.to_pickle('experiments/hyperparameter_tuning_results_keras.pkl')

In [64]:
fname = 'experiments/keras_models' + '_' + str(datetime.now().year) + '_' + str(datetime.now().month) \
    + '_' + str(datetime.now().day) + '_' + str(datetime.now().hour) + '_' + \
    str(datetime.now().minute) + '.txt'
    
eval_keras_params(fname = fname, param_grid = keras_grid)

finding best model parameters for fold number 1
Fitting 5 folds for each of 1296 candidates, totalling 6480 fits
Epoch 1/100
5/5 [==============================] - 1s 2ms/step - loss: 0.8499 - auc_10: 0.5251
Epoch 2/100
5/5 [==============================] - 0s 2ms/step - loss: 0.8320 - auc_10: 0.4827
Epoch 3/100
5/5 [==============================] - 0s 2ms/step - loss: 0.7479 - auc_10: 0.5272
Epoch 4/100
5/5 [==============================] - 0s 2ms/step - loss: 0.7151 - auc_10: 0.5690
Epoch 5/100
5/5 [==============================] - 0s 2ms/step - loss: 0.7294 - auc_10: 0.5265
Epoch 6/100
5/5 [==============================] - 0s 2ms/step - loss: 0.7159 - auc_10: 0.5438
Epoch 7/100
5/5 [==============================] - 0s 2ms/step - loss: 0.7222 - auc_10: 0.5278
Epoch 8/100
5/5 [==============================] - 0s 2ms/step - loss: 0.7102 - auc_10: 0.5336
Epoch 9/100
5/5 [==============================] - 0s 2ms/step - loss: 0.6977 - auc_10: 0.5618
Epoch 10/100
5/5 [==============

3/3 [==============================] - 0s 2ms/step - loss: 0.3243 - auc_11: 0.7892
Epoch 71/100
3/3 [==============================] - 0s 2ms/step - loss: 0.3353 - auc_11: 0.7813
Epoch 72/100
3/3 [==============================] - 0s 2ms/step - loss: 0.3178 - auc_11: 0.7967
Epoch 73/100
3/3 [==============================] - 0s 2ms/step - loss: 0.3168 - auc_11: 0.8132
Epoch 74/100
3/3 [==============================] - 0s 2ms/step - loss: 0.3378 - auc_11: 0.8044
Epoch 75/100
3/3 [==============================] - 0s 2ms/step - loss: 0.3183 - auc_11: 0.7978
Epoch 76/100
3/3 [==============================] - 0s 2ms/step - loss: 0.3065 - auc_11: 0.8339
Epoch 77/100
3/3 [==============================] - 0s 2ms/step - loss: 0.2992 - auc_11: 0.8419
Epoch 78/100
3/3 [==============================] - 0s 2ms/step - loss: 0.3218 - auc_11: 0.8145
Epoch 79/100
3/3 [==============================] - 0s 2ms/step - loss: 0.3153 - auc_11: 0.8131
Epoch 80/100
3/3 [==============================] - 0

5/5 [==============================] - 0s 4ms/step - loss: 0.5143 - auc_16: 0.8802
Epoch 3/100
5/5 [==============================] - 0s 4ms/step - loss: 0.4145 - auc_16: 0.9275
Epoch 4/100
5/5 [==============================] - 0s 3ms/step - loss: 0.3577 - auc_16: 0.9385
Epoch 5/100
5/5 [==============================] - 0s 3ms/step - loss: 0.2940 - auc_16: 0.9546
Epoch 6/100
5/5 [==============================] - 0s 3ms/step - loss: 0.2380 - auc_16: 0.9732
Epoch 7/100
5/5 [==============================] - 0s 3ms/step - loss: 0.2235 - auc_16: 0.9731
Epoch 8/100
5/5 [==============================] - 0s 3ms/step - loss: 0.1986 - auc_16: 0.9782
Epoch 9/100
5/5 [==============================] - 0s 3ms/step - loss: 0.1502 - auc_16: 0.9883
Epoch 10/100
5/5 [==============================] - 0s 3ms/step - loss: 0.1324 - auc_16: 0.9928
Epoch 11/100
5/5 [==============================] - 0s 4ms/step - loss: 0.1174 - auc_16: 0.9952
Epoch 12/100
5/5 [==============================] - 0s 3ms/s

3/3 [==============================] - 0s 3ms/step - loss: 0.0031 - auc_17: 1.0000
Epoch 72/100
3/3 [==============================] - 0s 4ms/step - loss: 0.0028 - auc_17: 1.0000
Epoch 73/100
3/3 [==============================] - 0s 3ms/step - loss: 0.0032 - auc_17: 1.0000
Epoch 74/100
3/3 [==============================] - 0s 3ms/step - loss: 0.0027 - auc_17: 1.0000
Epoch 75/100
3/3 [==============================] - 0s 4ms/step - loss: 0.0021 - auc_17: 1.0000
Epoch 76/100
3/3 [==============================] - 0s 4ms/step - loss: 0.0028 - auc_17: 1.0000
Epoch 77/100
3/3 [==============================] - 0s 4ms/step - loss: 0.0025 - auc_17: 1.0000
Epoch 78/100
3/3 [==============================] - 0s 4ms/step - loss: 0.0047 - auc_17: 1.0000
Epoch 79/100
3/3 [==============================] - 0s 3ms/step - loss: 0.0019 - auc_17: 1.0000
Epoch 80/100
3/3 [==============================] - 0s 3ms/step - loss: 0.0014 - auc_17: 1.0000
Epoch 81/100
3/3 [==============================] - 0

In [4]:
# best hyperparameters based on tuning

keras_params = {'layer_size_factor': 4, 
             'num_hidden_layers': 3,
             'dropout_rate': 0.4}
svm_params = {'C': 1, 
             'gamma': 0.0001, 
             'kernel': 'rbf', 
             'probability': True}
rf_params = {'max_depth': None,
            'max_features': 'auto', 
            'min_samples_leaf': 2, 
            'min_samples_split': 5, 
            'n_estimators': 140}
logreg_params = {'C': 0.1,
                'max_iter': 1000, 
                'penalty': 'l2'}

final_models = [
    ['svm', SVC(**svm_params)],
    ['rf', RandomForestClassifier(**rf_params)], 
    ['logreg', LogisticRegression(**logreg_params)]
]

In [44]:
# helper function to fit model and save it

def fit_save_model(name, model, X, y):
    print('fitting {}'.format(name))
    model.fit(X, y)
    fname = 'models/final_{}.sav'.format(name)
    pickle.dump(model, open(fname, 'wb'))

In [8]:
# fit final models on all training data and save

sm = SMOTE()
X_res, y_res = sm.fit_resample(X_train, y_train)

for name, model in final_models:
    fit_save_model(name = name, model = model, X = X_res, y = y_res)

In [9]:
# fit keras model on all training data and save
batch_size = 100
epochs = 50

final_mlp = build_model(input_shape = X_train.shape[1], **keras_params)
final_mlp.summary()
final_mlp.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = [AUC()])
final_mlp.fit(X_res, y_res, batch_size = batch_size, epochs = epochs)
final_mlp.save('models/final_mlp')

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 83)]              0         
                                                                 
 dense_4 (Dense)             (None, 16)                1344      
                                                                 
 dropout_3 (Dropout)         (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 16)                272       
                                                                 
 dropout_4 (Dropout)         (None, 16)                0         
                                                                 
 dense_6 (Dense)             (None, 16)                272       
                                                                 
 dropout_5 (Dropout)         (None, 16)                0   

## Evaluate ensemble models

In [19]:
# create ensemble models

mlp_ensemble = KerasClassifier(build_fn = build_model, input_shape = X_train.shape[1], **keras_params)
mlp_ensemble._estimator_type = 'classifier'
svm_ensemble = SVC(**svm_params)
rf_ensemble = RandomForestClassifier(**rf_params)
logreg_ensemble = LogisticRegression(**logreg_params)

ensemble1_params = [{'estimators': [('mlp', mlp_ensemble), 
                                    ('svm', svm_ensemble)], 
                     'voting': 'soft'}]
ensemble2_params = [{'estimators': [('mlp', mlp_ensemble), 
                                    ('rf', rf_ensemble)], 
                     'voting': 'soft'}]
ensemble3_params = [{'estimators': [('mlp', mlp_ensemble), 
                                    ('logreg', logreg_ensemble)],
                     'voting': 'soft'}]
ensemble4_params = [{'estimators': [('svm', svm_ensemble), 
                                    ('rf', rf_ensemble)], 
                     'voting': 'soft'}]
ensemble5_params = [{'estimators': [('svm', svm_ensemble),  
                                    ('logreg', logreg_ensemble)], 
                     'voting': 'soft'}]
ensemble6_params = [{'estimators': [('rf', rf_ensemble),
                                    ('logreg', logreg_ensemble)], 
                     'voting': 'soft'}]

ensemble_models = [
    ['Ensemble 1 (mlp/svm)', VotingClassifier, ensemble1_params],
    ['Ensemble 2 (mlp/rf)', VotingClassifier, ensemble2_params],
    ['Ensemble 3 (mlp/logreg)', VotingClassifier, ensemble3_params],
    ['Ensemble 4 (svm/rf)', VotingClassifier, ensemble4_params],
    ['Ensemble 5 (svm/logreg)', VotingClassifier, ensemble5_params],
    ['Ensemble 6 (rf/logreg)', VotingClassifier, ensemble6_params],
]

In [20]:
# evaluate ensemble models

scores = ['roc_auc', 'precision', 'recall', 'f1', 'accuracy']
fname = 'experiments/sklearn_ensembles' + '_' + str(datetime.now().year) + '_' + str(datetime.now().month) \
    + '_' + str(datetime.now().day) + '_' + str(datetime.now().hour) + '_' + \
    str(datetime.now().minute) + '.txt'
    
for score in scores:
    eval_models(models = ensemble_models, score = score, X_train = X_train, y_train = y_train, fname = fname)

/Users/haydnhoffman/ml/my_env/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
50 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/haydnhoffman/ml/my_env/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/haydnhoffman/ml/my_env/lib/python3.9/site-packages/imblearn/pipeline.py", line 272, in fit
    self._final_estimator.fit(Xt, yt, **fit_params_last_step)
  File "/Users/haydnhoffman/ml/my_env/lib/python3.9/site-packages/sklearn/ensemble/_voting.py", line 324, in fit

KeyboardInterrupt: 